# Statistical Model
## Approach
In this notebook, we build the statistical model (also called 'OpenBox') used to determine the possible risk increase by certain parameters present in an individual interested in purchasinng a firearm from a dealer.

The idea behind OpenBox is based on Bayesian statistics: we compare the probability of an individual presenting certain conditions knowing they have been involved in a mass shooting to the prevalence of the conditions in the general population. We look at the following conditions:
- mental illness
- employment status
- arrest, charge and conviction
- autism
- age bracket

The risk factor is calculated for each condition in each US state based on publicly-available data. Bayes' Theorem is used to calculate the risk increase:

$ P(S|C) = {P(C|S)*P(S) \over P(C)} =  R*P(S)$, where:
- $ P(S|C) $ is the probability of an individual being a shooter ($S$) knowing that they have condition $C$
- $ P(C|S) $ is the probability of an individual having condition $C$ knowing that they are a shooter ($S$)
- $ P(S) $ is the probability of any individual in the population of being a shooter ($S$)
- $ P(C) $ is the prevalence of condition $C$ in the general population, i.e. the probability of any individual of having the condition

The risk factor $R$ is defined as:

$R = {P(C|S) \over P(C)}$, with:
- $R = 1$ means no risk increase compared to a member of the general population picked at random
- $R > 1$ means more risk compared to any member of the general population
- $R < 1$ means less risk compared to any member of the general population
- The higher $R$ is, the more risk the individual poses

## Data Sources
- **Shooter information**: Peterson, J., & Densley, J. (2023). The Violence Project database of mass shootings in the United States (Version 7). https://www.theviolenceproject.org
- **Mental Illness Information**: States with the highest levels of mental health illness - NiceRx. https://worldpopulationreview.com/state-rankings/mental-health-statistics-by-state
- **Unemployment Data**: Unemployment Rates for States - U.S. Bureau of Labor Statistics (2023). https://worldpopulationreview.com/state-rankings/unemployment-rate-by-state
- **Arrests by State**: Federal Bureau of Investigation (2018). https://ucr.fbi.gov/crime-in-the-u.s/2018/crime-in-the-u.s.-2018/topic-pages/tables/table-69 (Data for Iowa based on 2019 figures due to lack of information in 2018)
- **Autism prevalence**: National Library of Medicine, J Autism Dev Disord. 2020 Dec; 50(12): 4258–4266. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9128411/table/T2/
- **Age and Population**: U.S. Census Bureau (2018-2021). Accessed through `census` Python module API


### Import Dependencies

In [35]:
import pandas as pd
from pathlib import Path
import sys

# Ignnore warning
import warnings
warnings.simplefilter(action='ignore')

# Local modules
sys.path.append("../Local_Modules/")
import codebook
from codebook import replace_code_by_value, get_distribution
from data_exploration import dataset_info

## Shooter information
Import the data to calculate $P(C|S)$

In [36]:
# Import shooter data
shooters_df = pd.read_csv(Path('../clean_data/clean_shooters.csv'))

shooter_profile = shooters_df[
        [
        'Mental Illness',
        'Employment Status',
        'Highest Level of Justice System Involvement',
        'Autism Spectrum'
        ]]

shooter_profile.head()

,Mental Illness,Employment Status,Highest Level of Justice System Involvement,Autism Spectrum
0,1,0,4,0
1,2,0,0,0
2,4,1,1,0
3,0,1,0,0
4,2,0,0,0


In [37]:
stat_df = get_distribution(shooter_profile, 'Autism Spectrum',0)
stat_df

,Autism Spectrum,Percent
0,181,93.782383
1,12,6.217617


### Probability of a shooter having a diagnosed mental illness
Note: shooters with an indication of psychiatric disorder but no diagnosis are assumed to have no mental illness

In [38]:
prob_shooter_with_mental_illness = shooter_profile.loc[(shooter_profile['Mental Illness'] != '0') & (shooter_profile['Mental Illness'] != '4'),'Mental Illness'].count()
prob_shooter_with_mental_illness = 100*prob_shooter_with_mental_illness/len(shooter_profile)
prob_shooter_with_mental_illness

45.07772020725388

### Probability of a shooter being unemployed
Note: only shooters that are surely not working in the dataset are considered 'unemployed'. I.e. when the employment is unknown, it is assumed that the shooter works.

In [39]:
prob_shooter_being_unemployed = shooter_profile.loc[shooter_profile['Employment Status'] == '0','Employment Status'].count()
prob_shooter_being_unemployed = 100*prob_shooter_being_unemployed/len(shooter_profile)
prob_shooter_being_unemployed

54.40414507772021

### Probability of a shooter having been arrested, charged or convicted for a previous crime

In [40]:
# Determine the number of shooters that have been arrested (2), charged (3) or convicted (4)
prob_shooter_being_arrested = shooter_profile.loc[\
    (shooter_profile['Highest Level of Justice System Involvement'] == 2) |\
        (shooter_profile['Highest Level of Justice System Involvement'] == 3) |\
        (shooter_profile['Highest Level of Justice System Involvement'] == 4)\
        ,'Highest Level of Justice System Involvement'].count()

# Divide the number of shooters that have been arrested (2), charged (3) or convicted (4) by the total number of shooters
prob_shooter_being_arrested = 100*prob_shooter_being_arrested/len(shooter_profile)
prob_shooter_being_arrested

52.84974093264249

### Probability of a shooter having autism

In [41]:
prob_shooter_being_autistic = shooter_profile.loc[shooter_profile['Autism Spectrum'] == 1,'Autism Spectrum'].count()
prob_shooter_being_autistic = 100*prob_shooter_being_autistic/len(shooter_profile)
prob_shooter_being_autistic

6.217616580310881

## Conditions in General Population
Import data to calculate $P(C)$

### Mental Illness

In [42]:
# Import mental illness data per state
csv = Path('../raw_data/mental-health-statistics-by-state-[updated-may-2023].csv')
mentalillness_df = pd.read_csv(csv)

# Change state name with 2-letter state code
for key in codebook.codes_states.keys():
    mentalillness_df.loc[mentalillness_df['state']==key,'state'] = codebook.codes_states[key]

# Rename columns and keep only state and mental illness rate columns
mentalillness_df = mentalillness_df.rename(columns={'state': 'State', 'RatesOfMentalIllness':'Mental_Illness_Rate'})
mentalillness_df = mentalillness_df[['State', 'Mental_Illness_Rate']]

In [43]:
mentalillness_df.head()

,State,Mental_Illness_Rate
0,UT,29.68
1,OR,27.33
2,WV,26.05
3,KS,26.02
4,OK,25.59


### Employment Status

In [44]:
# Import unemplyment data
csv = Path('../raw_data/unemployment-rate-by-state-[updated-august-2023].csv')
unemployment_df = pd.read_csv(csv)

# Change state name with 2-letter state code
for key in codebook.codes_states.keys():
    unemployment_df.loc[unemployment_df['state']==key,'state'] = codebook.codes_states[key]

# Keep only 2021 data to be in line with most recent census data
unemployment_df = unemployment_df[['state','unemploymentRateJuly2021']]
    
# Rename columns and keep only state and mental illness rate columns
unemployment_df = unemployment_df.rename(columns={'state': 'State', 'unemploymentRateJuly2021':'Unemployment_Rate'})
unemployment_df = unemployment_df[['State', 'Unemployment_Rate']]

In [45]:
unemployment_df.head()

,State,Unemployment_Rate
0,NV,6.6
1,DC,7.1
2,CA,7.4
3,DE,5.5
4,TX,5.6


### Arrest

In [46]:
# Import arrest data
csv = Path('../raw_data/FBI_2018_crimeByStates.csv')
arrests_df = pd.read_csv(csv)

# Change state name with 2-letter state code
for key in codebook.codes_states.keys():
    arrests_df.loc[arrests_df['State']==key,'State'] = codebook.codes_states[key]

# Delete empty rows
arrests_df = arrests_df.dropna(how='any')

# Recalculate Rate
arrests_df['Rate'] = 100*arrests_df['Arrests']/arrests_df['Population']

# Rename columns and keep only state and mental illness rate columns
arrests_df = arrests_df.rename(columns={'Rate':'Arrest_Rate'})
arrests_df = arrests_df[['State', 'Arrest_Rate']]

arrests_df.head()

,State,Arrest_Rate
0,AL,2.586075
1,AK,4.152213
2,AZ,3.580559
3,AR,3.989615
4,CA,2.763300


### Autism

In [47]:
# Import Autism Data
csv = Path('../clean_data/clean_autism.csv')
autism_df = pd.read_csv(csv)

# Keep only State and Prevalence column (rename)
autism_df = autism_df.rename(columns={'Prevalence':'Autism_Rate'})
autism_df = autism_df[['State', 'Autism_Rate']]


In [48]:
autism_df.head()

,State,Autism_Rate
0,AL,2.12
1,AK,2.19
2,AZ,2.29
3,AR,2.03
4,CA,2.36


### Merge State Information dataframes

In [57]:
state_info_df = pd.merge(mentalillness_df, unemployment_df, how='outer', on='State')
state_info_df = pd.merge(state_info_df, arrests_df, how='outer', on='State')
state_info_df = pd.merge(state_info_df, autism_df, how='outer', on='State')


state_info_df.head()

,State,Mental_Illness_Rate,Unemployment_Rate,Arrest_Rate,Autism_Rate
0,UT,29.68,2.7,3.318017,2.28
1,OR,27.33,5.1,2.988274,2.28
2,WV,26.05,5.1,1.617094,2.07
3,KS,26.02,3.4,1.127836,2.19
4,OK,25.59,3.9,2.562794,2.13


## Risk Factor Table

In [58]:
state_risks_df = state_info_df[:]

# Rename columns
state_risks_df = state_risks_df.rename(columns={
    'Mental_Illness_Rate': 'Mental_Illness_Risk',
    'Unemployment_Rate': 'Unemployment_Risk',
    'Arrest_Rate': 'Arrest_Risk',
    'Autism_Rate': 'Autism_Risk',
})

# Calculate risk factors
state_risks_df['Mental_Illness_Risk'] = prob_shooter_with_mental_illness/state_info_df['Mental_Illness_Rate']
state_risks_df['Unemployment_Risk'] =  prob_shooter_being_unemployed/state_info_df['Unemployment_Rate']
state_risks_df['Arrest_Risk'] =  prob_shooter_being_arrested/state_info_df['Arrest_Rate']
state_risks_df['Autism_Risk'] =  prob_shooter_being_autistic/state_info_df['Autism_Rate']

# Replace NaN by 1 (no risk increase)
state_risks_df = state_risks_df.fillna(1)

# Display Risk Factor for OpenBox
state_risks_df

,State,Mental_Illness_Risk,Unemployment_Risk,Arrest_Risk,Autism_Risk
0,UT,1.518791,20.149683,15.928111,2.727025
1,OR,1.649386,10.667479,17.685706,2.727025
2,WV,1.730431,10.667479,32.681924,3.003680
3,KS,1.732426,16.001219,46.859422,2.839094
4,OK,1.761537,13.949781,20.621922,2.919069
5,WA,1.767061,10.462336,23.226414,2.919069
6,ID,1.808897,15.112263,17.729182,2.852118
7,OH,1.853525,10.667479,28.282539,2.946738
8,RI,1.868894,9.067358,21.495162,2.775722
9,AZ,1.886887,11.102887,14.760194,2.715116
